In [1]:
import numpy as np
import pandas as pd

import gc
import random
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import joblib
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import datatable as dt

### Load Data

In [2]:
path = Path('/kaggle/input')
assert path.exists()

In [3]:
%%time

data_types_dict = {
#     'row_id': 'int64',
    'content_type_id': 'bool',
    'timestamp': 'int64',
    'user_id': 'int32', 
    'content_id': 'int16', 
    'task_container_id': 'int8',
    'answered_correctly': 'int8', 
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'bool'
}
target = 'answered_correctly'
train_df = dt.fread(path/'riiid-test-answer-prediction/train.csv', columns=set(data_types_dict.keys())).to_pandas()

CPU times: user 58.6 s, sys: 16.4 s, total: 1min 14s
Wall time: 1min 46s


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 8 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int32  
 3   content_type_id                 bool   
 4   task_container_id               int32  
 5   answered_correctly              int32  
 6   prior_question_elapsed_time     float64
 7   prior_question_had_explanation  object 
dtypes: bool(1), float64(1), int32(4), int64(1), object(1)
memory usage: 3.9+ GB


In [5]:
%%time

train_df = train_df[train_df.content_type_id == False]

#arrange by timestamp
train_df = train_df.sort_values(['timestamp'], ascending=True).reset_index(drop = True)

CPU times: user 32 s, sys: 8.55 s, total: 40.5 s
Wall time: 40.5 s


In [6]:
del train_df['timestamp']
del train_df['content_type_id']

### Pre-process

In [7]:
n_skill = train_df["content_id"].nunique()
print("number skills", n_skill)
joblib.dump(n_skill, "n_skill.pkl")

number skills 13523


['n_skill.pkl']

In [8]:
%%time

group = train_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (r['content_id'].values, r['answered_correctly'].values))

joblib.dump(group, "group.pkl")

del train_df

CPU times: user 58.8 s, sys: 6.23 s, total: 1min 5s
Wall time: 1min 5s
